In [1]:
import pandas as pd
import numpy as np
import nbimporter
import json
import re
import scipy.stats

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

from scripts.trans_vs_cis import trans_vs_cis

In [2]:
combined = pd.read_csv('../data/combined.csv')
combined

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (8,18,19,20,22,23,25,39,74,75,77,215,297,298,306,307,308,309,311,312,313) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,STUDYID,WEIGHT_CISGENDER_TRANSPOP,WEIGHT_CISGENDER,WEIGHT_TRANSPOP,GMETHOD_TYPE,SURVEYCOMPLETED,GRESPONDENT_DATE,GCENREG,RACE,...,ACE_SEP,ACE_INC,ACE_EMO_I,ACE_PHY_I,ACE_SEX_I,ACE_IPV_I,ACE_SUB_I,ACE_MEN_I,ACE_SEP_I,ACE_INC_I
0,0,151768927,0.022039,,.9861429333687,,0,26-APR-2016,1,6,...,0,0,1,0,0,0,0,0,0,0
1,1,152357242,0.008485,,.3796825110912,,0,07-APR-2016,3,6,...,0,0,0,0,0,0,0,1,0,0
2,2,152444055,0.015764,,.705381155014,,0,01-MAY-2016,3,6,...,0,0,0,0,0,0,0,1,0,0
3,3,152525272,0.035655,,1.5953975915909,,0,20-APR-2016,4,6,...,1,1,1,0,0,1,1,1,1,1
4,4,152894493,0.041802,,1.870422244072,,0,05-MAY-2016,2,8,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431,1431,197167223,6.299798,5.1374626159668,,SCAN,2,27-DEC-2018,4,3,...,0,0,1,1,0,1,1,1,0,0
1432,1432,197167224,0.581383,.4741154909134,,SCAN,2,15-JAN-2019,4,6,...,0,0,1,0,1,0,0,1,0,0
1433,1433,197168195,2.584832,2.1079211235046,,WEB,2,13-DEC-2018,1,6,...,0,0,1,1,1,1,0,1,0,0
1434,1434,197169207,0.545099,.4445266127586,,WEB,2,08-DEC-2018,4,6,...,0,0,0,0,0,0,0,0,0,0


In [80]:
demographics = combined[['AGE', 'TRANS_CIS', 'GEDUCATION', 
                         'POVERTYCAT_I', 'RACE_RECODE_CAT5', 'SEX']]
demographics['TRANS_CIS'] = demographics['TRANS_CIS'].map({1: "Trans", 2: "Cis"})
demographics['SEX'] = demographics['SEX'].map({1: "Female", 2: "Male"})
demographics

,AGE,TRANS_CIS,GEDUCATION,POVERTYCAT_I,RACE_RECODE_CAT5,SEX
0,65,Trans,4,4,1,Male
1,38,Trans,5,3,1,Female
2,25,Trans,3,4,1,Female
3,18,Trans,1,4,1,Female
4,30,Trans,4,4,3,Female
...,...,...,...,...,...,...
1431,19,Cis,4,3,3,Male
1432,70,Cis,4,4,1,Female
1433,51,Cis,2,1,1,Male
1434,53,Cis,5,4,1,Male


In [81]:
demographics['TRANS_CIS'].value_counts()

Cis      1162
Trans     274
Name: TRANS_CIS, dtype: int64

In [82]:
demographics.to_csv("demographics.csv")

# Hypothesis

(from a more scientific method standpoint)
Transgenders have worse healthcare access than cisgenders.

In [3]:
def pivot_table_count(df, q):
    counts = df.pivot_table(
        index='TRANS_CIS',
        columns=q,
        values='SEX',
        aggfunc='count'
    )
    counts['total'] = counts.sum(axis=1)
    return counts

In [4]:
def stratify_income(x):
    if x >= 12:
        return 'high'
    elif x >= 9:
        return 'middle'
    else:
        return 'low'

## HINC_I: Household income with imputation

In [5]:
hinc_i = combined[['TRANS_CIS', 'HINC_I', 'SEX']]
hinc_i

,TRANS_CIS,HINC_I,SEX
0,1,11,2
1,1,7,1
2,1,9,1
3,1,11,1
4,1,8,1
...,...,...,...
1431,2,11,2
1432,2,12,1
1433,2,0,2
1434,2,12,2


In [6]:
hinc_i_count = pivot_table_count(hinc_i, 'HINC_I')
hinc_i_count

HINC_I,0,1,2,3,4,5,6,7,8,9,10,11,12,13,total
TRANS_CIS,,,,,,,,,,,,,,,
1,6,15,16,24,25,9,12,27,24,29,16,25,29,17,274
2,15,15,32,55,47,53,42,85,84,95,121,157,186,175,1162


In [7]:
hinc_i_prop = hinc_i_count.copy().drop(columns='total')
for col in hinc_i_prop:
    hinc_i_prop[col] = hinc_i_prop[col] / hinc_i_count['total']
hinc_i_prop

HINC_I,0,1,2,3,4,5,6,7,8,9,10,11,12,13
TRANS_CIS,,,,,,,,,,,,,,
1,0.021898,0.054745,0.058394,0.087591,0.091241,0.032847,0.043796,0.09854,0.087591,0.105839,0.058394,0.091241,0.105839,0.062044
2,0.012909,0.012909,0.027539,0.047332,0.040448,0.045611,0.036145,0.07315,0.072289,0.081756,0.104131,0.135112,0.160069,0.150602


In [8]:
hinc_i_prop.T.to_csv('hinc_i.csv')

In [9]:
hinc_i_count.drop(columns='total').T.to_csv('hinc_i.csv')

## AGE
- possible groups:
    - 18-24
    - 25-34
    - 35-44
    - 45-54
    - 55-64
    - 65+

In [10]:
def group_age(x):
    if 18 <= x <= 24:
        return '18-24'
    if x <= 34:
        return '25-34'
    if x <= 44:
        return '35-44'
    if x <= 54:
        return '45-54'
    if x <= 64:
        return '55-64'
    else:
        return '65+'

In [11]:
age = combined[['TRANS_CIS', 'AGE', 'SEX']]
age

,TRANS_CIS,AGE,SEX
0,1,65,2
1,1,38,1
2,1,25,1
3,1,18,1
4,1,30,1
...,...,...,...
1431,2,19,2
1432,2,70,1
1433,2,51,2
1434,2,53,2


In [12]:
age.groupby('TRANS_CIS')['AGE'].agg(['mean', 'median', 'max', 'min'])

,mean,median,max,min
TRANS_CIS,,,,
1,39.361314,34.0,72,18
2,56.384682,60.0,72,18


In [13]:
age = age.copy()
age['age_group'] = age['AGE'].apply(group_age)
age

,TRANS_CIS,AGE,SEX,age_group
0,1,65,2,65+
1,1,38,1,35-44
2,1,25,1,25-34
3,1,18,1,18-24
4,1,30,1,25-34
...,...,...,...,...
1431,2,19,2,18-24
1432,2,70,1,65+
1433,2,51,2,45-54
1434,2,53,2,45-54


In [14]:
age_count = age.pivot_table(
    index='TRANS_CIS',
    columns='age_group',
    values='SEX',
    aggfunc='count'
)
age_count

age_group,18-24,25-34,35-44,45-54,55-64,65+
TRANS_CIS,,,,,,
1,72,68,35,32,38,29
2,48,83,135,171,257,468


In [15]:
age_prop = age_count.copy()
for col in age_prop.columns:
    age_prop[col] = age_prop[col] / age_count.sum(axis=1)
age_prop

age_group,18-24,25-34,35-44,45-54,55-64,65+
TRANS_CIS,,,,,,
1,0.262774,0.248175,0.127737,0.116788,0.138686,0.105839
2,0.041308,0.071429,0.116179,0.147160,0.221170,0.402754


In [16]:
age_melt = age_prop.T.reset_index().melt(id_vars='age_group')
age_melt

,age_group,TRANS_CIS,value
0,18-24,1,0.262774
1,25-34,1,0.248175
2,35-44,1,0.127737
3,45-54,1,0.116788
4,55-64,1,0.138686
5,65+,1,0.105839
6,18-24,2,0.041308
7,25-34,2,0.071429
8,35-44,2,0.116179
9,45-54,2,0.147160


In [19]:
fig = px.bar(age_melt, 
       x='age_group', 
       y='value', 
       color='TRANS_CIS', 
       barmode='group', 
       color_discrete_sequence=['#fca2bb', '#bababa'],
       title='Age Groups',
       text=np.round(age_melt['value'], 4))
fig.update_yaxes(title='proportion')
fig.update_xaxes(title='')
fig.update_traces(textposition='outside', textfont_size=9)

In [18]:
age_count.T.to_csv('age.csv')

## GENDER_IDENTITY or SEX

- a trans woman is more likely to face worse treatment than a trans man (due to women just not being as equally treated as men in the healthcare system)
- how to map trans women and females together; trans men and males together?

In [22]:
def man_woman(row):
    if row['GENDER_IDENTITY'] == 4:
        return 1
    if row['GENDER_IDENTITY'] == 3:
        return 2
    else:
        return row['SEX']

In [23]:
genders = combined[['GENDER_IDENTITY', 'SEX']].copy()
genders['genders'] = genders.apply(man_woman, axis=1)
genders

,GENDER_IDENTITY,SEX,genders
0,4,2,1
1,3,1,2
2,3,1,2
3,3,1,2
4,3,1,2
...,...,...,...
1431,1,2,2
1432,2,1,1
1433,1,2,2
1434,1,2,2


In [24]:
genders[genders['GENDER_IDENTITY'] == 5]['SEX'].value_counts()

1    51
2    25
Name: SEX, dtype: int64

In [25]:
genders['genders'].value_counts()

1    777
2    659
Name: genders, dtype: int64

## Q84: Overall, how satisfied are you with the healthcare you receive at this place?

- Mann-Whitney test
- check for variance (it's okay if not equal)
- Planned missing is due to the No and Missing values from Q82; if someone answere "Yes" to Q82, then they would be eligible to answer Q84

In [60]:
q84 = combined[['TRANS_CIS', 'Q84', 'SEX', 'Q82']]
# q84 = q84[q84['Q84'] != ' ']
# q84['Q84'] = q84['Q84'].astype(int)
# q84 = q84[q84['Q84'] != 7]
q84

,TRANS_CIS,Q84,SEX,Q82
0,1,7,2,
1,1,7,1,
2,1,7,1,
3,1,7,1,
4,1,7,1,
...,...,...,...,...
1431,2,1,2,2
1432,2,4,1,2
1433,2,7,2,1
1434,2,2,2,2


In [61]:
trans_vs_cis(q84, 'Q82')

In [28]:
q84_count = q84.pivot_table(
    index='TRANS_CIS',
    columns='Q84',
    values='SEX',
    aggfunc='count'
)
q84_count = q84_count.loc[:, 1:5]
q84_count

Q84,1,2,3,4,5
TRANS_CIS,,,,,
1,58,91,20,9,2
2,512,432,67,16,8


In [29]:
q84_prop = q84_count.copy()
for col in q84_prop.columns:
    q84_prop[col] = q84_prop[col] / q84_count.sum(axis=1)
q84_prop.T

TRANS_CIS,1,2
Q84,,
1,0.322222,0.494686
2,0.505556,0.417391
3,0.111111,0.064734
4,0.050000,0.015459
5,0.011111,0.007729


In [30]:
q84_prop.T.to_csv('q84.csv')

In [31]:
q84_count.T.to_csv('q84.csv')

## Q85: Do you have one person you think of as your personal doctor or healthcare provider?

In [32]:
q85 = combined[['TRANS_CIS', 'Q85', 'SEX']]
q85 = q85[q85['Q85'] != ' ']
q85

,TRANS_CIS,Q85,SEX
0,1,1,2
1,1,1,1
2,1,1,1
3,1,2,1
4,1,1,1
...,...,...,...
1431,2,1,2
1432,2,1,1
1433,2,1,2
1434,2,2,2


In [33]:
q85_count = q85.pivot_table(
    index='TRANS_CIS',
    columns='Q85',
    values='SEX',
    aggfunc='count'
)
q85_count

Q85,1,2
TRANS_CIS,,
1,187,86
2,932,206


In [34]:
q85_count.sum(axis=1)

TRANS_CIS
1     273
2    1138
dtype: int64

In [35]:
trans_vs_cis(q85, 'Q85')

### Q85 + stratified income

In [36]:
q85_inc = combined[['TRANS_CIS', 'HINC_I', 'Q85', 'SEX']]
q85_inc = q85_inc[q85_inc['Q85'] != ' ']
q85_inc

,TRANS_CIS,HINC_I,Q85,SEX
0,1,11,1,2
1,1,7,1,1
2,1,9,1,1
3,1,11,2,1
4,1,8,1,1
...,...,...,...,...
1431,2,11,1,2
1432,2,12,1,1
1433,2,0,1,2
1434,2,12,2,2


In [37]:
q85_inc['income_level'] = q85_inc['HINC_I'].apply(stratify_income)
q85_inc

,TRANS_CIS,HINC_I,Q85,SEX,income_level
0,1,11,1,2,middle
1,1,7,1,1,low
2,1,9,1,1,middle
3,1,11,2,1,middle
4,1,8,1,1,low
...,...,...,...,...,...
1431,2,11,1,2,middle
1432,2,12,1,1,high
1433,2,0,1,2,low
1434,2,12,2,2,high


In [38]:
q85_low_inc = q85_inc[q85_inc['income_level'] == 'low']
q85_mid_inc = q85_inc[q85_inc['income_level'] == 'middle']
q85_high_inc = q85_inc[q85_inc['income_level'] == 'high']

In [39]:
pivot_table_count(q85_low_inc, 'Q85')

Q85,1,2,total
TRANS_CIS,,,
1,103,54,157
2,328,90,418


In [40]:
pivot_table_count(q85_mid_inc, 'Q85')

Q85,1,2,total
TRANS_CIS,,,
1,52,18,70
2,304,66,370


In [41]:
pivot_table_count(q85_high_inc, 'Q85')

Q85,1,2,total
TRANS_CIS,,,
1,32,14,46
2,300,50,350


### Q85 + race

In [42]:
q85_race = combined[['TRANS_CIS', 'RACE_RECODE_CAT5', 'Q85', 'SEX']]
q85_race = q85_race[q85_race['Q85'] != ' ']
q85_race

,TRANS_CIS,RACE_RECODE_CAT5,Q85,SEX
0,1,1,1,2
1,1,1,1,1
2,1,1,1,1
3,1,1,2,1
4,1,3,1,1
...,...,...,...,...
1431,2,3,1,2
1432,2,1,1,1
1433,2,1,1,2
1434,2,1,2,2


In [43]:
q85_white = q85_race[q85_race['RACE_RECODE_CAT5'] == 1]
q85_black = q85_race[q85_race['RACE_RECODE_CAT5'] == 2]
q85_latino = q85_race[q85_race['RACE_RECODE_CAT5'] == 3]
q85_multi = q85_race[q85_race['RACE_RECODE_CAT5'] == 4]
q85_other = q85_race[q85_race['RACE_RECODE_CAT5'] == 5]

In [44]:
pivot_table_count(q85_white, 'Q85')

Q85,1,2,total
TRANS_CIS,,,
1,129,58,187
2,786,158,944


In [45]:
pivot_table_count(q85_black, 'Q85')

Q85,1,2,total
TRANS_CIS,,,
1,12,9,21
2,40,13,53


In [46]:
pivot_table_count(q85_latino, 'Q85')

Q85,1,2,total
TRANS_CIS,,,
1,17,8,25
2,42,20,62


In [47]:
pivot_table_count(q85_multi, 'Q85')

Q85,1,2,total
TRANS_CIS,,,
1,20,4,24
2,39,10,49


In [48]:
pivot_table_count(q85_other, 'Q85')

Q85,1,2,total
TRANS_CIS,,,
1,9,7,16
2,25,5,30


## Q88: Was there a time in the past 12 months when you needed to see a doctor but could not because of the cost?

In [49]:
q88 = combined[['TRANS_CIS', 'Q88', 'SEX']]
q88 = q88[q88['Q88'] != ' ']
q88

,TRANS_CIS,Q88,SEX
0,1,2,2
1,1,2,1
2,1,1,1
3,1,2,1
4,1,2,1
...,...,...,...
1431,2,2,2
1432,2,2,1
1433,2,2,2
1434,2,2,2


In [50]:
trans_vs_cis(q88, 'Q88')

In [51]:
q88_count = q88.pivot_table(
    index='TRANS_CIS',
    columns='Q88',
    values='SEX',
    aggfunc='count'
)
q88_count

Q88,1,2
TRANS_CIS,,
1,74,195
2,119,1014


In [52]:
q88_count.sum(axis=1)

TRANS_CIS
1     269
2    1133
dtype: int64

In [53]:
q88_prop = q88_count.copy()
for col in q88_prop.columns:
    q88_prop[col] = q88_prop[col] / q88_count.sum(axis=1)
q88_prop

Q88,1,2
TRANS_CIS,,
1,0.275093,0.724907
2,0.105031,0.894969


## Q88 (feature engineering for logistic regression)

In [66]:
q88 = combined[['TRANS_CIS', 'WEIGHT_CISGENDER_TRANSPOP', 'Q88', 'HINC_I', 'AGE', 'Q85',
                'RACE_RECODE_CAT5', 'Q93', 'SEX', 'GENDER_IDENTITY', 'POVERTYCAT_I', 'Q99',
                'Q200', 'Q205_I', 'GEDUCATION']]
q88 = q88[q88['Q88'] != ' ']
q88

,TRANS_CIS,WEIGHT_CISGENDER_TRANSPOP,Q88,HINC_I,AGE,Q85,RACE_RECODE_CAT5,Q93,SEX,GENDER_IDENTITY,POVERTYCAT_I,Q99,Q200,Q205_I,GEDUCATION
0,1,0.022039,2,11,65,1,1,3,2,4,4,2,2,2,4
1,1,0.008485,2,7,38,1,1,4,1,3,3,2,2,2,5
2,1,0.015764,1,9,25,1,1,4,1,3,4,1,2,2,3
3,1,0.035655,2,11,18,2,1,3,1,3,4,2,2,4,1
4,1,0.041802,2,8,30,1,3,4,1,3,4,2,2,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431,2,6.299798,2,11,19,1,3,5,2,1,3,2,2,5,4
1432,2,0.581383,2,12,70,1,1,3,1,2,4,1,1,2,4
1433,2,2.584832,2,0,51,1,1,3,2,1,1,2,2,1,2
1434,2,0.545099,2,12,53,2,1,4,2,1,4,2,2,2,5


In [67]:
q88['Q85'].value_counts()

1    1111
2     285
        6
Name: Q85, dtype: int64

In [68]:
q88_cat = ['Q88', 'SEX', 'HINC_I', 'RACE_RECODE_CAT5', 'Q85', 'Q93', 'POVERTYCAT_I', 'Q99', 'Q200', 'Q205_I', 'GEDUCATION']
for i in q88_cat:
    trans_vs_cis(q88, i).show()

In [69]:
HINC_I_means = {
    0: 0,
    1: 2_500,
    2: 7_500,
    3: 12_500,
    4: 17_500,
    5: 22_500,
    6: 27_500,
    7: 35_000,
    8: 45_000,
    9: 55_000,
    10: 67_500,
    11: 82_500,
    12: 125_000,
    13: 150_000
}
q88['HINC_I_means'] = q88['HINC_I'].map(HINC_I_means)
q88['HINC_I_means'] = (q88['HINC_I_means'] - np.mean(q88['HINC_I_means'])) / np.std(q88['HINC_I_means'])
q88['HINC_I_strat'] = q88['HINC_I'].apply(stratify_income)
q88['HINC_I_strat'] = q88['HINC_I_strat'].map({'low': 0, 'middle': 1, 'high': 2})
q88['TRANS_CIS'] = q88['TRANS_CIS'].map({1: 1, 2: 0})
q88['Q88'] = q88['Q88'].map({'1': 1, '2': 0})
q88 = q88[q88['Q85'] != ' ']
q88['Q85'] = q88['Q85'].map({'1': 1, '2': 0})
q88['AGE'] = (q88['AGE'] - np.mean(q88['AGE'])) / np.std(q88['AGE'])
q88 = q88[q88['Q93'] != ' ']
q88 = q88[q88['Q99'] != ' ']
q88 = q88[q88['GEDUCATION'] != ' ']
q88 = q88[q88['Q200'] != ' ']
q88['Q93'] = q88['Q93'].astype(int)
q88['Q99'] = q88['Q99'].map({'1': 1, '2': 0})
q88['Q200'] = q88['Q200'].map({'1': 1, '2': 0})
q88['CURRENT_SEX'] = q88.apply(man_woman, axis=1)
q88['SEX'] = q88['SEX'].map({1: 1, 2: 0})
q88['CURRENT_SEX'] = q88['CURRENT_SEX'].map({1: 1, 2: 0})
q88

,TRANS_CIS,WEIGHT_CISGENDER_TRANSPOP,Q88,HINC_I,AGE,Q85,RACE_RECODE_CAT5,Q93,SEX,GENDER_IDENTITY,POVERTYCAT_I,Q99,Q200,Q205_I,GEDUCATION,HINC_I_means,HINC_I_strat,CURRENT_SEX
0,1,0.022039,0,11,0.696200,1,1,3,0,4,4,0,0,2,4,0.261983,1,1
1,1,0.008485,0,7,-0.907588,1,1,4,1,3,3,0,0,2,5,-0.737687,0,0
2,1,0.015764,1,9,-1.679782,1,1,4,1,3,4,1,0,2,3,-0.316774,1,0
3,1,0.035655,0,11,-2.095579,0,1,3,1,3,4,0,0,4,1,0.261983,1,0
4,1,0.041802,0,8,-1.382784,1,3,4,1,3,4,0,0,1,4,-0.527230,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431,0,6.299798,0,11,-2.036180,1,3,5,0,1,3,0,0,5,4,0.261983,1,0
1432,0,0.581383,0,12,0.993198,1,1,3,1,2,4,1,1,2,4,1.156424,2,1
1433,0,2.584832,0,0,-0.135394,1,1,3,0,1,1,0,0,1,2,-1.474286,0,0
1434,0,0.545099,0,12,-0.016594,0,1,4,0,1,4,0,0,2,5,1.156424,2,0


In [70]:
q88_corr = q88.corr()

q88_corr_fig = ff.create_annotated_heatmap(
    x = q88_corr.columns.tolist(),
    y = q88_corr.columns.tolist(),
    z = np.array(q88_corr),
    zmin=-1,
    zmax=1,
    annotation_text = np.around(np.array(q88_corr), decimals=2),
    hoverinfo='z',
    colorscale='balance',
    showscale=True
)
q88_corr_fig.update_layout(
    title_text = 'Correlation Matrix',
    title_x = 0.5,
    title_y = 1,
    width = 1000,
    height = 1000,
    yaxis_autorange='reversed'
)
q88_corr_fig.show()

features highly correlated with each other: 
- TRANS_CIS, GENDER_IDENTITY
- HINC_I, POVERTYCAT_I
- SEX, CURRENT_SEX
- GENDER_IDENTITY, CURRENT_SEX
- TRANS_CIS, AGE
- TRANS_CIS, WEIGHT_CISGENDER_TRANSPOP
- GENDER_IDENTITY, WEIGHT_CISGENDER_TRANSPOP
- GENDER_IDENTITY, AGE
- HINC_I, Q93

features highly correlated with response variable (Q88):
- 

In [71]:
q88.to_csv('q88.csv')

### Q88 + stratified income
- testing mediator: income
- low: <50K (0-8)
- middle: 50K to 150K (9-12)
- high: >150K (13)


- after stratifying with the above method, had to change high to >100K (12-13) due to 0 Transgenders and 4 Cisgenders answering Yes (this makes sense, as they are high income); does >100K still constitute high income? 
- btw, the >150K metric is from online sources; however since this survey was conducted form 2016-2018, could >100K still be considered high income?

In [56]:
q88_inc = combined[['TRANS_CIS', 'HINC_I', 'Q88', 'SEX']]
q88_inc = q88_inc[q88_inc['Q88'] != ' ']
q88_inc

,TRANS_CIS,HINC_I,Q88,SEX
0,1,11,2,2
1,1,7,2,1
2,1,9,1,1
3,1,11,2,1
4,1,8,2,1
...,...,...,...,...
1431,2,11,2,2
1432,2,12,2,1
1433,2,0,2,2
1434,2,12,2,2


In [57]:
q88_inc['HINC_I'].value_counts()

12    206
13    185
11    179
10    136
9     123
7     109
8     105
3      76
4      70
5      61
6      54
2      47
1      30
0      21
Name: HINC_I, dtype: int64

In [58]:
q88_inc = q88_inc.copy()
q88_inc['income_level'] = q88_inc['HINC_I'].apply(stratify_income)
q88_inc

,TRANS_CIS,HINC_I,Q88,SEX,income_level
0,1,11,2,2,middle
1,1,7,2,1,low
2,1,9,1,1,middle
3,1,11,2,1,middle
4,1,8,2,1,low
...,...,...,...,...,...
1431,2,11,2,2,middle
1432,2,12,2,1,high
1433,2,0,2,2,low
1434,2,12,2,2,high


In [59]:
q88_low_inc = q88_inc[q88_inc['income_level'] == 'low']
q88_mid_inc = q88_inc[q88_inc['income_level'] == 'middle']
q88_high_inc = q88_inc[q88_inc['income_level'] == 'high'][['TRANS_CIS', 'Q88', 'SEX']]

In [60]:
q88_high_inc_count = pivot_table_count(q88_high_inc, 'Q88')
q88_high_inc_count

Q88,1,2,total
TRANS_CIS,,,
1,5,40,45
2,13,333,346


In [61]:
q88_mid_inc_count = pivot_table_count(q88_mid_inc, 'Q88')
q88_mid_inc_count

Q88,1,2,total
TRANS_CIS,,,
1,14,56,70
2,30,338,368


In [62]:
q88_low_inc_count = pivot_table_count(q88_low_inc, 'Q88')
q88_low_inc_count

Q88,1,2,total
TRANS_CIS,,,
1,55,99,154
2,76,343,419


## Q93: Would you say that in general your health is…

In [63]:
q93 = combined[['TRANS_CIS', 'Q93', 'SEX']]
q93_missing = q93[q93['Q93'] == ' '].value_counts()
q93 = q93[q93['Q93'] != ' ']
q93

,TRANS_CIS,Q93,SEX
0,1,3,2
1,1,4,1
2,1,4,1
3,1,3,1
4,1,4,1
...,...,...,...
1431,2,5,2
1432,2,3,1
1433,2,3,2
1434,2,4,2


In [64]:
trans_vs_cis(q93, 'Q93')

In [65]:
q93_count = q93.pivot_table(
    index='TRANS_CIS',
    columns='Q93',
    values='SEX',
    aggfunc='count'
)
q93_count

Q93,1,2,3,4,5
TRANS_CIS,,,,,
1,19,50,103,74,28
2,31,148,419,432,111


In [66]:
q93_prop = q93_count.copy()
for col in q93_prop.columns:
    q93_prop[col] = q93_prop[col] / q93_count.sum(axis=1)
q93_prop.T

TRANS_CIS,1,2
Q93,,
1,0.069343,0.027169
2,0.182482,0.129711
3,0.375912,0.367222
4,0.270073,0.378615
5,0.102190,0.097283


In [67]:
q93_prop.T.to_csv('q93.csv')

In [68]:
q93_count.T.to_csv('q93.csv')